In [ ]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [ ]:
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [ ]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [ ]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [ ]:
#COURSE
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

In [ ]:
course_data_records = course_data_df.to_dict('records')
course_data_records

In [ ]:
for rec in course_data_records:

    record = {
        'ID_Course__c': rec['ID_Course'],
        'CourseCode__c': rec['CourseCode'],
        'CourseName__c': rec['CourseName'],
        'CreditHours__c': rec['CreditHours'],
        'BootCampCourse__c': rec['BootCampCourse'],
    }
    
    try:
        sf.Course__C.create(record)
    except Exception as e:
        print(e)

In [ ]:
course_lookup_list = []
data = sf.query_all_iter("SELECT ID_Course__c, CourseName__c, Name FROM Course__c")
for row in data:
    rec = {
        'ID_Course': row['ID_Course__c'],
        'CourseName': row['CourseName__c'],
        'CourseSF': row['Name']
    }
    course_lookup_list.append(rec)

In [ ]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

In [ ]:
#CLASS
query = '''
    SELECT
        c.*
        
    FROM
        class c
        
'''
class_data_df = pd.read_sql(query, conn)
class_data_df.head()

In [ ]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
class_data_df.head()

In [ ]:
class_data_df['StartDate'] = pd.to_datetime(class_data_df['StartDate']).dt.date
class_data_df['EndDate'] = pd.to_datetime(class_data_df['EndDate']).dt.date
class_data_df.head()

In [ ]:
for index, row in class_data_df.iterrows():
        record = {
        'ID_Class__c': row['ID_Class'],
        'ID_Course__c' : row['ID_Course'],
        'Section__c': row['Section'],
        'StartDate__c': str(row['StartDate']),
        'EndDate__c': str(row['EndDate']),
        'Course_Name__c': row['CourseSF']
    }
    
        try:
            sf.Class__c.create(record)
        except Exception as e:
            print(e)

In [ ]:
#delete
#class_records = sf.query("SELECT Id FROM Staff__c")
#recs_to_delete = [{'Id': r['Id']} for r in class_records['records']]
#recs_to_delete

In [ ]:
#delete 
#sf.bulk.Staff__c.delete(recs_to_delete)